In [ ]:
# importando bibliotecas
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# instancia do problema

# demandas dos clientes
d = [1, 1, 1, 1, 1, 1]

# custo fixo dos depósitos
f = [21, 16, 11, 24]

# custo de transporte
c = [[6, 2, 3, 4],
     [1, 9, 4, 11],
     [15, 2, 6, 3],
     [9, 11, 4, 8],
     [7, 23, 2, 9],
     [4, 3, 1, 5]
    ]

# range dos clientes e depósitos
M = range(len(d))
N = range(len(f))

In [ ]:
# Modelo MIP do facility location problem

# criando modelo
mip = gp.Model("facility")

# variável y: y[p] == 1 se a depósito p é aberto, 0 caso contrário
y = []
for j in N:
    y.append(mip.addVar(vtype=GRB.BINARY, obj=f[j], name="y[%d]" % j))

# variável x: x[w,p] é quantidade transportada do depósito p para o cliente w
x = []
for i in M:
    x.append([])
    for j in N:
        x[i].append(mip.addVar(obj=c[i][j], name="x[%d,%d]" % (i, j)))

# problema de mininização
mip.modelSense = GRB.MINIMIZE

# restrição de demanda
for i in M:
    mip.addConstr(sum(x[i][j] for j in N) == d[i], "dem[%d]" % i)

# restrição de linke entre x e y
for j in N:
    mip.addConstr(sum(x[i][j] for i in M) <= len(d) * y[j], "cap[%d]" % j)

# escrevendo o modelo
mip.write('flp_mip.lp')

# usando o método barreira para resolver a relaxação na raiz
mip.Params.method = 2

# resolve o problema
mip.optimize()

In [ ]:
# imprime a solução do MIP
print('\nCusto total: %g' % mip.objVal)

print('\n Solução:')
for j in N:
    if y[j].x > 0.99:
        print('\nDepósito %s é aberto' % j)
        for i in M:
            if x[i][j].x > 0:
                print('Transporte de %g unidades para o cliente %s' % (x[i][j].x, i))
    else:
        print('\nDepósito %s é fechada!' % j)

In [ ]:
# relaxando a variáveis inteiras do MIP
for v in mip.getVars():
    if v.vType != GRB.CONTINUOUS:
        v.vType = GRB.CONTINUOUS

mip.update()

mip.write('flp_lp.lp')

# resolve o problema
mip.optimize()

In [ ]:
# imprime a solução do MIP
print('\nCusto total: %g' % mip.objVal)

print('\n Solução:')
for j in N:
    if y[j].x > 0.0:
        print('\nDeposito %s é aberto' % j)
        for i in M:
            if x[i][j].x > 0:
                print('Transporte de %g unidades para o cliente %s' % (x[i][j].x, i))
    else:
        print('\nDepósito %s é fechada!' % j)

In [ ]:
# Reformula o facility location problem e resolve a relaxação linear do problema

# criando modelo
hconv = gp.Model("flp_hconv")

# variável y: y[p] == 1 se a fábrica p é aberta, 0 caso contrário
y = []
for j in N:
    y.append(hconv.addVar(vtype=GRB.BINARY, obj=f[j], name="y[%d]" % j))

# variável x: x[w,p] é quantidade transportada da fábrica p para o armazém w
x = []
for i in M:
    x.append([])
    for j in N:
        x[i].append(hconv.addVar(obj=c[i][j], name="x[%d,%d]" % (i, j)))

# problema de mininização
hconv.modelSense = GRB.MINIMIZE

# restrição de demanda
for i in M:
    hconv.addConstr(sum(x[i][j] for j in N) == d[i], "dem[%d]" % i)

# restrição de link entre x e y
for i in M:
    for j in N:
        hconv.addConstr(x[i][j] <= y[j], "cap[%d]" % j)

# escrevendo o modelo
hconv.write('hconv_flp_mip.lp')

# relaxando a variáveis inteiras
for v in hconv.getVars():
    if v.vType != GRB.CONTINUOUS:
        v.vType = GRB.CONTINUOUS
        
# escrevendo o modelo
hconv.write('hconv_flp_lp.lp')

# resolve o problema
hconv.optimize()

In [ ]:
# imprime a solução do modelo hconv
print('\nCusto total: %g' % hconv.objVal)

print('\n Solução:')
for j in N:
    if y[j].x > 0.99:
        print('\nFábrica %s é aberto' % j)
        for i in M:
            if x[i][j].x > 0:
                print('Transporte de %g unidades para o armazém %s' % (x[i][j].x, i))
    else:
        print('\nFábrica %s é fechada!' % j)